In [1]:
import sys
import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb

from catboost import CatBoostRegressor

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.kernel_ridge import KernelRidge

In [2]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
train   = pd.read_csv('train_cleaning.csv')
test    = pd.read_csv('test_cleaning.csv')
y_train = pd.read_csv('y_train.csv')

In [4]:
y_train = y_train.values.ravel()

In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(train, y_train, test_size = 0.2, random_state=42)

In [6]:
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train)  
    rmse= np.sqrt(-cross_val_score(model, train, y_train, cv = kf, scoring="neg_mean_squared_error"))
    return(rmse)

### Kernel Ridge

In [7]:
KR = make_pipeline(RobustScaler(), KernelRidge(
         alpha= 0.9826,
         kernel= 'polynomial',
         degree= 2,
         coef0= 3.5
        ))           

KR_score = rmsle_cv(KR)
print("Kernel Ridge score: {:.4f} ({:.4f})".format(KR_score.mean(), KR_score.std()))

Kernel Ridge score: 0.1191 (0.0152)


### Elastic Net

In [8]:
ENet = make_pipeline(RobustScaler(), ElasticNet(
        alpha= 0.0002835,
        l1_ratio= 0.9999905,
        random_state= 0 
        ))

ENet_score = rmsle_cv(ENet)
print("ENet score: {:.4f} ({:.4f})".format(ENet_score.mean(), ENet_score.std()))

ENet score: 0.1130 (0.0136)


### Lasso

In [9]:
Lass = make_pipeline(RobustScaler(), Lasso(
        max_iter=3000,
        alpha= 0.000281,
        random_state= 5 
        )) 

Lass_score = rmsle_cv(Lass)
print("Lass score: {:.4f} ({:.4f})".format(Lass_score.mean(), Lass_score.std()))


Lass score: 0.1130 (0.0136)


### Gradient Boosting

In [10]:
Graboost = GradientBoostingRegressor(
    n_estimators= 3000,
    max_depth= 4,
    min_samples_leaf= 2,
    max_features= 'sqrt',
    loss='huber',
    random_state= 0,
    learning_rate= 0.0223,
    min_samples_split= 100 
    )

Graboost_score = rmsle_cv(Graboost)
print("Graboost score: {:.4f} ({:.4f})".format(Graboost_score.mean(), Graboost_score.std()))

Graboost score: 0.1097 (0.0169)


### Ridge

In [11]:
RidgeR = make_pipeline(RobustScaler(), Ridge(
    alpha= 8.8435,
    solver= 'cholesky'
        ))    
                       
RidgeR_score = rmsle_cv(RidgeR)
print("RidgeR score: {:.4f} ({:.4f})".format(RidgeR_score.mean(), RidgeR_score.std()))

RidgeR score: 0.1151 (0.0133)


### XGBoost

In [12]:
XGBoost = xgb.XGBRegressor(
    learning_rate= 0.0225,
    colsample_bytree= 0.1,
    n_estimators= 2500,
    max_depth= 3,
    subsample= 0.6982,
    min_child_weight=1, 
    )

XGBoost_score = rmsle_cv(XGBoost)
print("XGBoost score: {:.4f} ({:.4f})".format(XGBoost_score.mean(), XGBoost_score.std()))

XGBoost score: 0.1094 (0.0159)


### Bayesian Ridge

In [13]:
BRidge = make_pipeline(RobustScaler(), BayesianRidge(
        n_iter= 100,
        alpha_1= 6e-05,
        alpha_2= 0.01,
        lambda_1= 0.01,
        lambda_2= 6e-05,
        tol= 1e-05
        ))
                       
BRidge_score = rmsle_cv(BRidge)
print("BRidge score: {:.4f} ({:.4f})".format(BRidge_score.mean(), BRidge_score.std()))

BRidge score: 0.1152 (0.0133)


### LightGBM

In [14]:
LGB = lgb.LGBMRegressor(
    learning_rate= 0.0166,
    objective= 'regression',
    num_leaves= 30,
    num_iterations= 1500,
    feature_fraction = 0.2175,
    bagging_fraction= .9870,
    min_data_in_leaf= 9,
    max_depth= 68
)

LGB_score = rmsle_cv(LGB)
print("LGB score: {:.4f} ({:.4f})".format(LGB_score.mean(), LGB_score.std()))

LGB score: 0.1175 (0.0165)


### CatBoost

In [15]:
CatB = make_pipeline(RobustScaler(), CatBoostRegressor(
    iterations= 1929,
    depth= 2, 
    learning_rate= 0.0229,
    loss_function= 'RMSE',
    l2_leaf_reg=0,
    rsm= 1.0,
    one_hot_max_size=194
))

CatB_score = rmsle_cv(CatB)
print("CatB score: {:.4f} ({:.4f})".format(CatB_score.mean(), CatB_score.std()))

0:	learn: 11.7582666	total: 52.6ms	remaining: 1m 41s
1:	learn: 11.4893127	total: 54.7ms	remaining: 52.7s
2:	learn: 11.2264961	total: 56.3ms	remaining: 36.2s
3:	learn: 10.9697377	total: 58.2ms	remaining: 28s
4:	learn: 10.7188648	total: 59.8ms	remaining: 23s
5:	learn: 10.4737473	total: 61.8ms	remaining: 19.8s
6:	learn: 10.2342115	total: 63.4ms	remaining: 17.4s
7:	learn: 10.0001037	total: 65ms	remaining: 15.6s
8:	learn: 9.7714469	total: 66.4ms	remaining: 14.2s
9:	learn: 9.5479483	total: 68.1ms	remaining: 13.1s
10:	learn: 9.3296748	total: 69.7ms	remaining: 12.2s
11:	learn: 9.1164044	total: 71.1ms	remaining: 11.4s
12:	learn: 8.9080139	total: 72.5ms	remaining: 10.7s
13:	learn: 8.7044173	total: 74ms	remaining: 10.1s
14:	learn: 8.5054963	total: 75.3ms	remaining: 9.61s
15:	learn: 8.3110940	total: 76.5ms	remaining: 9.15s
16:	learn: 8.1211814	total: 77.9ms	remaining: 8.77s
17:	learn: 7.9356424	total: 79.1ms	remaining: 8.4s
18:	learn: 7.7543595	total: 80.5ms	remaining: 8.09s
19:	learn: 7.5772403	t

165:	learn: 0.3555137	total: 296ms	remaining: 3.14s
166:	learn: 0.3501492	total: 298ms	remaining: 3.14s
167:	learn: 0.3446026	total: 299ms	remaining: 3.14s
168:	learn: 0.3393741	total: 301ms	remaining: 3.14s
169:	learn: 0.3346073	total: 303ms	remaining: 3.14s
170:	learn: 0.3299689	total: 305ms	remaining: 3.13s
171:	learn: 0.3251744	total: 307ms	remaining: 3.14s
172:	learn: 0.3205644	total: 309ms	remaining: 3.14s
173:	learn: 0.3158195	total: 310ms	remaining: 3.13s
174:	learn: 0.3117318	total: 312ms	remaining: 3.13s
175:	learn: 0.3074586	total: 314ms	remaining: 3.12s
176:	learn: 0.3032791	total: 315ms	remaining: 3.12s
177:	learn: 0.2995982	total: 317ms	remaining: 3.12s
178:	learn: 0.2955434	total: 319ms	remaining: 3.12s
179:	learn: 0.2921451	total: 322ms	remaining: 3.13s
180:	learn: 0.2885840	total: 324ms	remaining: 3.12s
181:	learn: 0.2846162	total: 325ms	remaining: 3.12s
182:	learn: 0.2810660	total: 327ms	remaining: 3.12s
183:	learn: 0.2775725	total: 329ms	remaining: 3.12s
184:	learn: 

403:	learn: 0.1265436	total: 630ms	remaining: 2.38s
404:	learn: 0.1264090	total: 632ms	remaining: 2.38s
405:	learn: 0.1262476	total: 635ms	remaining: 2.38s
406:	learn: 0.1262005	total: 637ms	remaining: 2.38s
407:	learn: 0.1261486	total: 639ms	remaining: 2.38s
408:	learn: 0.1261266	total: 640ms	remaining: 2.38s
409:	learn: 0.1259927	total: 642ms	remaining: 2.38s
410:	learn: 0.1259058	total: 644ms	remaining: 2.38s
411:	learn: 0.1258517	total: 646ms	remaining: 2.38s
412:	learn: 0.1257407	total: 648ms	remaining: 2.38s
413:	learn: 0.1255222	total: 649ms	remaining: 2.38s
414:	learn: 0.1254756	total: 651ms	remaining: 2.37s
415:	learn: 0.1254589	total: 653ms	remaining: 2.37s
416:	learn: 0.1254185	total: 654ms	remaining: 2.37s
417:	learn: 0.1253834	total: 655ms	remaining: 2.37s
418:	learn: 0.1251788	total: 657ms	remaining: 2.37s
419:	learn: 0.1251447	total: 658ms	remaining: 2.36s
420:	learn: 0.1250140	total: 659ms	remaining: 2.36s
421:	learn: 0.1249815	total: 660ms	remaining: 2.36s
422:	learn: 

576:	learn: 0.1168664	total: 872ms	remaining: 2.04s
577:	learn: 0.1168503	total: 874ms	remaining: 2.04s
578:	learn: 0.1167842	total: 875ms	remaining: 2.04s
579:	learn: 0.1167841	total: 876ms	remaining: 2.04s
580:	learn: 0.1166635	total: 877ms	remaining: 2.04s
581:	learn: 0.1166436	total: 879ms	remaining: 2.03s
582:	learn: 0.1165741	total: 880ms	remaining: 2.03s
583:	learn: 0.1165739	total: 881ms	remaining: 2.03s
584:	learn: 0.1164871	total: 883ms	remaining: 2.03s
585:	learn: 0.1164825	total: 884ms	remaining: 2.03s
586:	learn: 0.1164824	total: 886ms	remaining: 2.02s
587:	learn: 0.1164693	total: 887ms	remaining: 2.02s
588:	learn: 0.1164422	total: 889ms	remaining: 2.02s
589:	learn: 0.1164366	total: 890ms	remaining: 2.02s
590:	learn: 0.1163850	total: 892ms	remaining: 2.02s
591:	learn: 0.1163697	total: 895ms	remaining: 2.02s
592:	learn: 0.1163167	total: 897ms	remaining: 2.02s
593:	learn: 0.1163164	total: 898ms	remaining: 2.02s
594:	learn: 0.1163073	total: 899ms	remaining: 2.02s
595:	learn: 

807:	learn: 0.1113051	total: 1.2s	remaining: 1.67s
808:	learn: 0.1112199	total: 1.21s	remaining: 1.67s
809:	learn: 0.1112177	total: 1.21s	remaining: 1.67s
810:	learn: 0.1112093	total: 1.21s	remaining: 1.67s
811:	learn: 0.1111529	total: 1.21s	remaining: 1.67s
812:	learn: 0.1111205	total: 1.21s	remaining: 1.67s
813:	learn: 0.1111186	total: 1.22s	remaining: 1.67s
814:	learn: 0.1110369	total: 1.22s	remaining: 1.67s
815:	learn: 0.1110346	total: 1.23s	remaining: 1.67s
816:	learn: 0.1110329	total: 1.23s	remaining: 1.67s
817:	learn: 0.1110294	total: 1.23s	remaining: 1.67s
818:	learn: 0.1109433	total: 1.23s	remaining: 1.67s
819:	learn: 0.1109371	total: 1.23s	remaining: 1.67s
820:	learn: 0.1108938	total: 1.23s	remaining: 1.67s
821:	learn: 0.1108931	total: 1.24s	remaining: 1.66s
822:	learn: 0.1108918	total: 1.24s	remaining: 1.66s
823:	learn: 0.1108418	total: 1.24s	remaining: 1.66s
824:	learn: 0.1108273	total: 1.24s	remaining: 1.66s
825:	learn: 0.1108257	total: 1.24s	remaining: 1.66s
826:	learn: 0

...

827:	learn: 0.1115820	total: 1.34s	remaining: 1.78s
828:	learn: 0.1115816	total: 1.34s	remaining: 1.78s
829:	learn: 0.1115229	total: 1.34s	remaining: 1.78s
830:	learn: 0.1115224	total: 1.34s	remaining: 1.78s
831:	learn: 0.1114894	total: 1.35s	remaining: 1.78s
832:	learn: 0.1114510	total: 1.35s	remaining: 1.77s
833:	learn: 0.1114505	total: 1.35s	remaining: 1.77s
834:	learn: 0.1114196	total: 1.35s	remaining: 1.77s
835:	learn: 0.1113622	total: 1.35s	remaining: 1.77s
836:	learn: 0.1113332	total: 1.36s	remaining: 1.77s
837:	learn: 0.1113134	total: 1.36s	remaining: 1.77s
838:	learn: 0.1113130	total: 1.36s	remaining: 1.77s
839:	learn: 0.1113126	total: 1.36s	remaining: 1.77s
840:	learn: 0.1113121	total: 1.36s	remaining: 1.77s
841:	learn: 0.1112452	total: 1.37s	remaining: 1.77s
842:	learn: 0.1111927	total: 1.37s	remaining: 1.76s
843:	learn: 0.1111395	total: 1.37s	remaining: 1.76s
844:	learn: 0.1111213	total: 1.37s	remaining: 1.76s
845:	learn: 0.1110874	total: 1.38s	remaining: 1.76s
846:	learn: 

1055:	learn: 0.1078835	total: 1.72s	remaining: 1.42s
1056:	learn: 0.1078779	total: 1.72s	remaining: 1.42s
1057:	learn: 0.1078614	total: 1.72s	remaining: 1.42s
1058:	learn: 0.1078080	total: 1.72s	remaining: 1.42s
1059:	learn: 0.1077985	total: 1.73s	remaining: 1.41s
1060:	learn: 0.1077629	total: 1.73s	remaining: 1.41s
1061:	learn: 0.1077607	total: 1.73s	remaining: 1.41s
1062:	learn: 0.1076811	total: 1.73s	remaining: 1.41s
1063:	learn: 0.1076608	total: 1.73s	remaining: 1.41s
1064:	learn: 0.1076592	total: 1.73s	remaining: 1.41s
1065:	learn: 0.1076591	total: 1.74s	remaining: 1.41s
1066:	learn: 0.1076559	total: 1.74s	remaining: 1.4s
1067:	learn: 0.1076469	total: 1.74s	remaining: 1.4s
1068:	learn: 0.1076468	total: 1.74s	remaining: 1.4s
1069:	learn: 0.1076429	total: 1.74s	remaining: 1.4s
1070:	learn: 0.1075816	total: 1.75s	remaining: 1.4s
1071:	learn: 0.1075769	total: 1.75s	remaining: 1.4s
1072:	learn: 0.1075723	total: 1.75s	remaining: 1.4s
1073:	learn: 0.1075165	total: 1.75s	remaining: 1.39s


1286:	learn: 0.1044149	total: 2.09s	remaining: 1.04s
1287:	learn: 0.1043684	total: 2.1s	remaining: 1.04s
1288:	learn: 0.1043607	total: 2.1s	remaining: 1.04s
1289:	learn: 0.1043607	total: 2.1s	remaining: 1.04s
1290:	learn: 0.1043186	total: 2.1s	remaining: 1.04s
1291:	learn: 0.1043084	total: 2.1s	remaining: 1.04s
1292:	learn: 0.1043050	total: 2.11s	remaining: 1.04s
1293:	learn: 0.1043050	total: 2.11s	remaining: 1.03s
1294:	learn: 0.1043030	total: 2.11s	remaining: 1.03s
1295:	learn: 0.1042872	total: 2.11s	remaining: 1.03s
1296:	learn: 0.1042183	total: 2.12s	remaining: 1.03s
1297:	learn: 0.1042168	total: 2.12s	remaining: 1.03s
1298:	learn: 0.1042109	total: 2.12s	remaining: 1.03s
1299:	learn: 0.1042068	total: 2.12s	remaining: 1.03s
1300:	learn: 0.1042032	total: 2.12s	remaining: 1.02s
1301:	learn: 0.1041996	total: 2.12s	remaining: 1.02s
1302:	learn: 0.1041996	total: 2.13s	remaining: 1.02s
1303:	learn: 0.1041887	total: 2.13s	remaining: 1.02s
1304:	learn: 0.1041849	total: 2.13s	remaining: 1.02

1516:	learn: 0.1023035	total: 2.47s	remaining: 670ms
1517:	learn: 0.1023024	total: 2.47s	remaining: 669ms
1518:	learn: 0.1023019	total: 2.47s	remaining: 667ms
1519:	learn: 0.1022956	total: 2.47s	remaining: 666ms
1520:	learn: 0.1022921	total: 2.48s	remaining: 664ms
1521:	learn: 0.1022639	total: 2.48s	remaining: 663ms
1522:	learn: 0.1022639	total: 2.48s	remaining: 661ms
1523:	learn: 0.1022357	total: 2.48s	remaining: 660ms
1524:	learn: 0.1022320	total: 2.48s	remaining: 658ms
1525:	learn: 0.1022162	total: 2.49s	remaining: 657ms
1526:	learn: 0.1022143	total: 2.49s	remaining: 655ms
1527:	learn: 0.1022110	total: 2.49s	remaining: 654ms
1528:	learn: 0.1022078	total: 2.49s	remaining: 652ms
1529:	learn: 0.1022076	total: 2.49s	remaining: 651ms
1530:	learn: 0.1021797	total: 2.5s	remaining: 649ms
1531:	learn: 0.1021770	total: 2.5s	remaining: 647ms
1532:	learn: 0.1021736	total: 2.5s	remaining: 646ms
1533:	learn: 0.1021703	total: 2.5s	remaining: 644ms
1534:	learn: 0.1021703	total: 2.5s	remaining: 643m

1744:	learn: 0.1007228	total: 2.84s	remaining: 299ms
1745:	learn: 0.1007186	total: 2.84s	remaining: 298ms
1746:	learn: 0.1006990	total: 2.84s	remaining: 296ms
1747:	learn: 0.1006938	total: 2.85s	remaining: 295ms
1748:	learn: 0.1006936	total: 2.85s	remaining: 293ms
1749:	learn: 0.1006917	total: 2.85s	remaining: 291ms
1750:	learn: 0.1006906	total: 2.85s	remaining: 290ms
1751:	learn: 0.1006885	total: 2.85s	remaining: 288ms
1752:	learn: 0.1006706	total: 2.85s	remaining: 287ms
1753:	learn: 0.1006434	total: 2.86s	remaining: 285ms
1754:	learn: 0.1006434	total: 2.86s	remaining: 283ms
1755:	learn: 0.1006418	total: 2.86s	remaining: 282ms
1756:	learn: 0.1006229	total: 2.86s	remaining: 280ms
1757:	learn: 0.1006154	total: 2.86s	remaining: 279ms
1758:	learn: 0.1006125	total: 2.87s	remaining: 277ms
1759:	learn: 0.1006096	total: 2.87s	remaining: 275ms
1760:	learn: 0.1005891	total: 2.87s	remaining: 274ms
1761:	learn: 0.1005790	total: 2.87s	remaining: 272ms
1762:	learn: 0.1005142	total: 2.87s	remaining:

### Stacking

In [16]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    #cloning models to fit the data
    def fit(self, X_train, Y_train):
        self.models_ = [clone(x) for x in self.models]
        
        #training the cloned models
        for model in self.models_:
            model.fit(X_train.values, Y_train)
            
        return self
    
    #predicting based on the cloned models
    def predict(self, X_val):
        predictions = np.column_stack([
            model.predict(X_val) for model in self.models_])
        #averaging the predictions
        return np.mean(predictions, axis=1)

### Averaged  models score

In [17]:
# stack models together
averaged_models = AveragingModels(models = (ENet, Lass, Graboost, RidgeR))

In [18]:
# compute score of stacked models
score = rmsle_cv(averaged_models)
print(" Averaged models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged models score: 0.1094 (0.0145)



In [19]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### Stacked Model Prediction Score

In [21]:
averaged_models.fit(X_train, Y_train)
averaged_train_pred = averaged_models.predict(X_val)
# stacked_predict = np.exp(averaged_train_pred)
print(rmse(Y_val, averaged_train_pred))

0.11569591056976032


### XGBoost Model Prediction Score

In [22]:
XGBoost.fit(X_val, Y_val)
XGBoost_train_predict = XGBoost.predict(X_val)
# XGBoost_predict = np.exp(XGBoost_train_predict)
print(rmse(Y_val, XGBoost_train_predict))

0.00922292184357076


### Light GBM Model Prediction Score

In [23]:
LGB.fit(X_val, Y_val)
LGB_train_predict = LGB.predict(X_val)
# LGB_predict = np.exp(LGB_train_predict)
print(rmse(Y_val, LGB_train_predict))

0.006380033897874042


### CatB Model Prediction Score

In [24]:
CatB.fit(X_val, Y_val)
CatB_train_predict = CatB.predict(X_val)
# XGBoost_predict = np.exp(CatB_train_predict)
print(rmse(Y_val, CatB_train_predict))

0:	learn: 11.7582656	total: 1.9ms	remaining: 3.67s
1:	learn: 11.4893087	total: 3.74ms	remaining: 3.61s
2:	learn: 11.2264900	total: 5.27ms	remaining: 3.38s
3:	learn: 10.9697423	total: 6.51ms	remaining: 3.13s
4:	learn: 10.7188699	total: 7.34ms	remaining: 2.82s
5:	learn: 10.4737566	total: 8.26ms	remaining: 2.65s
6:	learn: 10.2342694	total: 9.04ms	remaining: 2.48s
7:	learn: 10.0002671	total: 9.8ms	remaining: 2.35s
8:	learn: 9.7716350	total: 10.6ms	remaining: 2.26s
9:	learn: 9.5482187	total: 11.4ms	remaining: 2.18s
10:	learn: 9.3299588	total: 12.3ms	remaining: 2.15s
11:	learn: 9.1167021	total: 13.1ms	remaining: 2.09s
12:	learn: 8.9083351	total: 13.9ms	remaining: 2.05s
13:	learn: 8.7046867	total: 14.7ms	remaining: 2.01s
14:	learn: 8.5057433	total: 15.5ms	remaining: 1.97s
15:	learn: 8.3113303	total: 16.2ms	remaining: 1.94s
16:	learn: 8.1214372	total: 17ms	remaining: 1.91s
17:	learn: 7.9359078	total: 17.6ms	remaining: 1.87s
18:	learn: 7.7546355	total: 18.4ms	remaining: 1.84s
19:	learn: 7.57752

242:	learn: 0.2142535	total: 174ms	remaining: 1.21s
243:	learn: 0.2136349	total: 175ms	remaining: 1.21s
244:	learn: 0.2123861	total: 175ms	remaining: 1.21s
245:	learn: 0.2110119	total: 176ms	remaining: 1.21s
246:	learn: 0.2107868	total: 177ms	remaining: 1.2s
247:	learn: 0.2098827	total: 178ms	remaining: 1.2s
248:	learn: 0.2088042	total: 178ms	remaining: 1.2s
249:	learn: 0.2076399	total: 179ms	remaining: 1.2s
250:	learn: 0.2069403	total: 180ms	remaining: 1.2s
251:	learn: 0.2058663	total: 181ms	remaining: 1.2s
252:	learn: 0.2049966	total: 182ms	remaining: 1.2s
253:	learn: 0.2035423	total: 183ms	remaining: 1.2s
254:	learn: 0.2029836	total: 183ms	remaining: 1.2s
255:	learn: 0.2019146	total: 184ms	remaining: 1.2s
256:	learn: 0.2012623	total: 185ms	remaining: 1.2s
257:	learn: 0.2009166	total: 186ms	remaining: 1.2s
258:	learn: 0.2004665	total: 186ms	remaining: 1.2s
259:	learn: 0.2003309	total: 187ms	remaining: 1.2s
260:	learn: 0.2002154	total: 188ms	remaining: 1.2s
261:	learn: 0.1999796	total

469:	learn: 0.1580697	total: 360ms	remaining: 1.12s
470:	learn: 0.1580669	total: 361ms	remaining: 1.12s
471:	learn: 0.1580632	total: 362ms	remaining: 1.12s
472:	learn: 0.1580619	total: 362ms	remaining: 1.11s
473:	learn: 0.1573883	total: 363ms	remaining: 1.11s
474:	learn: 0.1573868	total: 364ms	remaining: 1.11s
475:	learn: 0.1565956	total: 365ms	remaining: 1.11s
476:	learn: 0.1562762	total: 366ms	remaining: 1.11s
477:	learn: 0.1562749	total: 367ms	remaining: 1.11s
478:	learn: 0.1562736	total: 368ms	remaining: 1.11s
479:	learn: 0.1562304	total: 369ms	remaining: 1.11s
480:	learn: 0.1562255	total: 370ms	remaining: 1.11s
481:	learn: 0.1562214	total: 371ms	remaining: 1.11s
482:	learn: 0.1562164	total: 371ms	remaining: 1.11s
483:	learn: 0.1558182	total: 372ms	remaining: 1.11s
484:	learn: 0.1558142	total: 373ms	remaining: 1.11s
485:	learn: 0.1558132	total: 374ms	remaining: 1.11s
486:	learn: 0.1558068	total: 375ms	remaining: 1.11s
487:	learn: 0.1558057	total: 376ms	remaining: 1.11s
488:	learn: 

720:	learn: 0.1412591	total: 547ms	remaining: 916ms
721:	learn: 0.1408607	total: 548ms	remaining: 916ms
722:	learn: 0.1408351	total: 549ms	remaining: 915ms
723:	learn: 0.1408316	total: 550ms	remaining: 915ms
724:	learn: 0.1408072	total: 551ms	remaining: 914ms
725:	learn: 0.1408054	total: 551ms	remaining: 914ms
726:	learn: 0.1408025	total: 552ms	remaining: 913ms
727:	learn: 0.1407792	total: 553ms	remaining: 912ms
728:	learn: 0.1407569	total: 554ms	remaining: 912ms
729:	learn: 0.1407293	total: 555ms	remaining: 911ms
730:	learn: 0.1407261	total: 556ms	remaining: 911ms
731:	learn: 0.1407255	total: 557ms	remaining: 910ms
732:	learn: 0.1407250	total: 557ms	remaining: 909ms
733:	learn: 0.1406906	total: 558ms	remaining: 909ms
734:	learn: 0.1404811	total: 559ms	remaining: 908ms
735:	learn: 0.1404598	total: 560ms	remaining: 908ms
736:	learn: 0.1404394	total: 561ms	remaining: 907ms
737:	learn: 0.1404356	total: 562ms	remaining: 906ms
738:	learn: 0.1404351	total: 562ms	remaining: 906ms
739:	learn: 

975:	learn: 0.1312363	total: 730ms	remaining: 713ms
976:	learn: 0.1311033	total: 731ms	remaining: 712ms
977:	learn: 0.1311032	total: 731ms	remaining: 711ms
978:	learn: 0.1311031	total: 733ms	remaining: 711ms
979:	learn: 0.1310944	total: 733ms	remaining: 710ms
980:	learn: 0.1310700	total: 734ms	remaining: 710ms
981:	learn: 0.1310699	total: 735ms	remaining: 709ms
982:	learn: 0.1310685	total: 736ms	remaining: 708ms
983:	learn: 0.1310672	total: 737ms	remaining: 708ms
984:	learn: 0.1307103	total: 738ms	remaining: 707ms
985:	learn: 0.1307102	total: 739ms	remaining: 707ms
986:	learn: 0.1307089	total: 740ms	remaining: 706ms
987:	learn: 0.1303401	total: 741ms	remaining: 705ms
988:	learn: 0.1303400	total: 742ms	remaining: 705ms
989:	learn: 0.1303319	total: 742ms	remaining: 704ms
990:	learn: 0.1303076	total: 743ms	remaining: 704ms
991:	learn: 0.1303065	total: 744ms	remaining: 703ms
992:	learn: 0.1302833	total: 745ms	remaining: 702ms
993:	learn: 0.1302756	total: 746ms	remaining: 702ms
994:	learn: 

1222:	learn: 0.1227304	total: 913ms	remaining: 527ms
1223:	learn: 0.1227300	total: 914ms	remaining: 526ms
1224:	learn: 0.1227266	total: 915ms	remaining: 526ms
1225:	learn: 0.1227233	total: 916ms	remaining: 525ms
1226:	learn: 0.1227233	total: 917ms	remaining: 524ms
1227:	learn: 0.1227233	total: 917ms	remaining: 524ms
1228:	learn: 0.1227233	total: 918ms	remaining: 523ms
1229:	learn: 0.1227110	total: 919ms	remaining: 522ms
1230:	learn: 0.1227080	total: 920ms	remaining: 522ms
1231:	learn: 0.1226349	total: 921ms	remaining: 521ms
1232:	learn: 0.1226349	total: 922ms	remaining: 520ms
1233:	learn: 0.1226320	total: 923ms	remaining: 520ms
1234:	learn: 0.1226320	total: 924ms	remaining: 519ms
1235:	learn: 0.1226315	total: 926ms	remaining: 519ms
1236:	learn: 0.1226315	total: 927ms	remaining: 518ms
1237:	learn: 0.1226311	total: 928ms	remaining: 518ms
1238:	learn: 0.1226307	total: 929ms	remaining: 517ms
1239:	learn: 0.1226303	total: 930ms	remaining: 517ms
1240:	learn: 0.1226303	total: 931ms	remaining:

1442:	learn: 0.1197575	total: 1.28s	remaining: 430ms
1443:	learn: 0.1197545	total: 1.28s	remaining: 429ms
1444:	learn: 0.1195667	total: 1.28s	remaining: 429ms
1445:	learn: 0.1195665	total: 1.28s	remaining: 428ms
1446:	learn: 0.1195661	total: 1.28s	remaining: 427ms
1447:	learn: 0.1195658	total: 1.29s	remaining: 427ms
1448:	learn: 0.1195618	total: 1.29s	remaining: 427ms
1449:	learn: 0.1195615	total: 1.29s	remaining: 426ms
1450:	learn: 0.1195268	total: 1.29s	remaining: 425ms
1451:	learn: 0.1195264	total: 1.29s	remaining: 424ms
1452:	learn: 0.1195262	total: 1.29s	remaining: 424ms
1453:	learn: 0.1195262	total: 1.29s	remaining: 423ms
1454:	learn: 0.1195259	total: 1.29s	remaining: 422ms
1455:	learn: 0.1195256	total: 1.29s	remaining: 421ms
1456:	learn: 0.1193908	total: 1.3s	remaining: 420ms
1457:	learn: 0.1193875	total: 1.3s	remaining: 419ms
1458:	learn: 0.1193875	total: 1.3s	remaining: 419ms
1459:	learn: 0.1193875	total: 1.3s	remaining: 418ms
1460:	learn: 0.1193872	total: 1.3s	remaining: 417m

1810:	learn: 0.1183079	total: 1.64s	remaining: 107ms
1811:	learn: 0.1183079	total: 1.64s	remaining: 106ms
1812:	learn: 0.1183079	total: 1.64s	remaining: 105ms
1813:	learn: 0.1183079	total: 1.64s	remaining: 104ms
1814:	learn: 0.1183079	total: 1.64s	remaining: 103ms
1815:	learn: 0.1183079	total: 1.64s	remaining: 102ms
1816:	learn: 0.1183079	total: 1.64s	remaining: 101ms
1817:	learn: 0.1183078	total: 1.64s	remaining: 100ms
1818:	learn: 0.1183067	total: 1.64s	remaining: 99.4ms
1819:	learn: 0.1183055	total: 1.64s	remaining: 98.5ms
1820:	learn: 0.1183055	total: 1.65s	remaining: 97.6ms
1821:	learn: 0.1183044	total: 1.65s	remaining: 96.7ms
1822:	learn: 0.1183044	total: 1.65s	remaining: 95.8ms
1823:	learn: 0.1183044	total: 1.65s	remaining: 94.9ms
1824:	learn: 0.1183044	total: 1.65s	remaining: 94ms
1825:	learn: 0.1183044	total: 1.65s	remaining: 93.1ms
1826:	learn: 0.1183044	total: 1.65s	remaining: 92.2ms
1827:	learn: 0.1183044	total: 1.65s	remaining: 91.2ms
1828:	learn: 0.1183034	total: 1.65s	re

### RMSE on the entire training set

In [27]:
overall_stack = XGBoost_train_predict * 0.45 + LGB_train_predict * 0.5 + averaged_train_pred * 0.05

In [29]:
print( "RMSE on the entire training dataset: {}".format(
    rmse(Y_val, overall_stack)
))

RMSE on the entire training dataset: 0.01007045630039962
